# Create statistics for a given scenario run

This notebook explains how to create statistics on the overall workflow by running the ```make_statistics``` Snakemake rule.

1. Start by building the PyPSA-Earth [tutorial model](https://pypsa-earth.readthedocs.io/en/latest/short_tutorial.html). 
2. A test case for Nigeria ("NG") has been set up in the ```config.NG.yaml``` file in the ```configs/scenarios``` [folder](https://github.com/javier-cp6/pypsa-earth/tree/main/configs/scenarios).
3. Run the command ```snakemake -j 1 make_statistics```, which will trigger the ```make_statistics.py``` script. As a result, the file ```results/stats.csv``` (shown in the table below) will be created with relevant information, such as:  

    - For clean_osm_data and download_osm_data: the number of elements, length of the lines and length of DC lines are stored.
    - For build_shapes: the surface, total GDP, total population and number of shapes are collected.
    - For build_renewable_profiles: total available potential and average production are collected.
    - For network rules (base_network, add_electricity, simplify_network and solve_network): length of lines, number of buses and total installed capacity by generation technology.
    - Execution time for the rules, when benchmark is available.


In [1]:
import pandas as pd
from scripts._helpers import read_csv_nafix

In [2]:
df = read_csv_nafix(str("./results/NG/stats.csv"), header=[0, 1], index_col=0)
df = df.T

In [3]:
# add unit column and set as index
df.loc[df.index.get_level_values(1).str.contains('size'), 'unit'] = 'EA'
df.loc[df.index.get_level_values(1).str.contains('length'), 'unit'] = 'meter'
df = df.set_index('unit', append=True)

In [4]:
# format length values to nine decimal places
mask = df.index.get_level_values(1).str.contains('length')
df.loc[mask, 'NG'] = df.loc[mask, 'NG'].map('{:,.9f}'.format)

In [5]:
df.style.format(precision=2, thousands=",")